* Funding rate arbitrage on Bybit. Note that we only long/short perpetual futures in the code. As we only profit from the funding rate. We solely care about the long/short position of perpetual futures. Including long/short position of spot is useless.

* The backtesting period includes the whole period of perpetual futures up to 2024-09-10 00:00:00. (10/9)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from typing import Optional
import csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

#Data import

In [ ]:
perps = {
    "BTC": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/BTCUSDTPERP200325160000-240910000000.csv'),
    "ETH": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/ETHUSDTPERP201021080000-240910000000.csv'),
    "NEIRO": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/NEIROETHUSDTPERP240815080000-240910000000.csv'),
    "SOL": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/SOLUSDTPERP210629080000-240910000000.csv'),
    "1000PEPE": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/1000PEPEUSDTPERP230503080000-240910000000.csv'),
    "DOGS": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/DOGSUSDTPERP240826080000-240910000000.csv'),
    "SUI": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/SUIUSDTPERP230503080000-240910000000.csv'),
    "WIF": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/WIFUSDTPERP240111080000-240910000000.csv'),
    "POPCAT": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/POPCATUSDTPERP240328080000-240910000000.csv'),
    "DOGE": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/DOGEUSDTPERP210602160000-240910000000.csv'),
    "FTM": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/FTMUSDTPERP210923080000-240910000000.csv'),
    "XRP": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/XRPUSDTPERP210513160000-240910000000.csv'),
    "TON": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/TONUSDTPERP230831000000-240910000000.csv'),
    "ORDI": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/ORDIUSDTPERP230522000000-240910000000.csv'),
    "AAVE": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/AAVEUSDTPERP210513160000-240910000000.csv'),
    "AVAX": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/AVAXUSDTPERP210915160000-240910000000.csv'),
    "NOT": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/NOTUSDTPERP240516150000-240910000000.csv'),
    "PEOPLE": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/PEOPLEUSDTPERP211229160000-240910000000.csv'),
    "LINK": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/LINKUSDTPERP201021080000-240910000000.csv'),
    "SEI": pd.read_csv('/content/drive/MyDrive/Crypto/Strategies/Data/Fund_rate/SEIUSDTPERP230816000000-240910000000.csv')
}

#Backtesting

In [ ]:
# check if consecutive days in window are +ve/-ve
def check_signs(window: int):
    if all(x > 0 for x in window):
        return -1  # if consecutive days in window are +ve => short perp
    elif all(x < 0 for x in window):
        return 1  # if consecutive days in window are -ve => long perp
    else:
        return 0 # if neither => no position

In [ ]:
# backtesting

def backtesting(df: pd.DataFrame, window: int, plot: bool = False) -> Optional[pd.Series]:

    df = df.copy()
    df['perp_pos'] = df['funding_rate'].rolling(window).apply(check_signs).shift(1).fillna(0)
    df['perp_pos_t+1'] = df['perp_pos'].shift(-1)
    df['trade'] = np.where(
                    df['perp_pos_t+1'].isna() & df['perp_pos'].notna(), # if previous day is nan + today is not nan => we do two trades (long/short perp + short/long spot)
                    2,
                    np.where(
                        df['perp_pos_t+1'].notna() & df['perp_pos'].notna(), # previous day and today are both not nan
                        abs(df['perp_pos_t+1'] - df['perp_pos']) * 2, # check if pervious day and today have different positions
                                                                      # e.g. if previous day is -1 & today is 0 => we do two trades as closing short perp + long spot positions
                        np.nan # if previous day and today are both nan, trade is nan. This happens to days in initial window as we start rolling()
                    )
                )

    df['pnl'] =  - (df['funding_rate'] * df['perp_pos']) - df['trade']*0.05/100
    df['cumu'] = df['pnl'].cumsum()
    df['dd'] = df['cumu'].cummax() - df['cumu']

    annual_return = round(df['pnl'].mean() * 3 * 365, 2)

    if df['pnl'].std() != 0:
        sharpe = round((df['pnl'].mean() / df['pnl'].std()) * np.sqrt(3 * 365), 2)
    else:
        sharpe = np.nan

    mdd = round(df['dd'].max(), 5)

    # avoid division of zero
    if mdd != 0:
        calmar = round(annual_return / mdd,2)
    else:
        calmar = np.nan

    if plot:
      print(pd.Series([window, sharpe, calmar, annual_return, mdd],index= ['window', 'sharpe', 'calmar', 'annual_return', 'mdd']))
      fig = px.line(df,x='datetime', y=['cumu'])
      fig.show()
      return

    return pd.Series([window, sharpe, calmar, annual_return, mdd],index= ['window', 'sharpe', 'calmar', 'annual_return', 'mdd'])


In [ ]:
# # optimization
# with open('frarb_result.csv', 'w', newline='') as file:

#     writer = csv.writer(file)

#     writer.writerows([["token", "window", "sharpe", "calmar", "annual_return", "mdd"]])

#     for token, _ in perps.items():

#         windows = np.arange(2,42,2)

#         result_list=[]

#         for window in windows:
#             result_list.append(backtesting(df = perps[token], window = window))

#         result_df = pd.DataFrame(result_list)
#         result_df = result_df.sort_values(by = 'annual_return',ascending=False)
#         result_df.iloc[0]

#         writer.writerows([[token] + result_df.iloc[0].to_list()])



# Result

https://docs.google.com/spreadsheets/d/1ZRFBfnqZbdu2wK4wwzo90M5eMLnQAied7wZtK3AUm2Y/edit?usp=sharing
